# Async state machine

So, how does async work technically.
To achieve a similar flow as Promises DotNet took a different direction.  
It's a much more mechanical solution involving state machines and lowering, as opposed to callback mechanics.

Consider the following code, from the earlier example:
```ts
import { waitFor } from "./helpfiles/Wait";

async function waitingExample() : Promise<void>
{
    console.log("Start of method");
    await waitFor(600);
    console.log("End of method");
} 

void Promise.resolve(waitingExample());
```
If we translate this to C# it will look something like this:

In [2]:
using System;
using System.Threading.Tasks;

async Task WaitingExample() {

    Console.WriteLine("Start of method");
    await Task.Delay(600);
    Console.WriteLine("End of method");
}

await WaitingExample();

Start of method
End of method


Remember that functionally this is just split into 2 methods at the await.  
However, technically it looks very different.  
  
If you lower this code, see [this example](https://sharplab.io/#gist:e25ac27df8564525d2229bcb89bf0520) on sharplab.io
you will recognize `MoveNext()` twice.  
**Surprise** it works kind of similar to enumerators.  

Because we have no Program class with Main function the example generates a lot of code for us, however we are interested in: `<<<Main>$>g__WaitingExample|0_0>d`.  
In this example the state machine has 4 states:  
- `-2`: Completed  
  Note that this is set for both exceptions and completions
- `-1`: Task Running
- `0`: Awaiting finished  
  ```csharp
    if (!awaiter.IsCompleted)
    {
        num = (<>1__state = 0);
        //...
    }
  ```

If you were to add another await you'll add states here.  
For example:  
```csharp
async Task WaitingExample() {

    Console.WriteLine("Start of method");
    await Task.Delay(300);
    Console.WriteLine("Middle of method");
    await Task.Delay(300);
    Console.WriteLine("End of method");
}
```
Add's a `1` state, apparently using goto's.  
The more awaits your method has, the more states added to this machine.  
It probably changes the way it optimizes the loop when this object becomes larger, just like with if and switch statements but that is something you can play around with to validate.  

The way it eventually runs this is by calling `<Main>$(args).GetAwaiter().GetResult();`.  
This is handled by the compiler for you, when using Console apps by generating this, when using web apps the framework takes care of this.
